In [1]:
import pandas as pd
import json

f= open('data/grafo_autores.json') 
data_autores = json.load(f)
f.close()

f= open('data/lista_papers.json')
data_papers = json.load(f)
f.close()

In [2]:
from pyvis.network import Network

autor_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, cdn_resources='in_line')

inicio = []
final = []
peso = []

min_articulos = 10
for autor , autor_data in data_autores.items():
    if data_autores[autor]['n_articulos'] > min_articulos:
        for coautores, n in autor_data['coautores'].items():
            if data_autores[coautores]['n_articulos'] > min_articulos:
                # print(f'Autor: {autor}, coautor con: {coautores} en {n} articulos')
                inicio.append(autor)
                final.append(coautores)
                peso.append(n)

edge_data = zip(inicio, final, peso)

for e in edge_data:
                src = e[0]
                dst = e[1]
                w = e[2]

                autor_net.add_node(src, title=src)
                autor_net.add_node(dst, title=dst)
                autor_net.add_edge(src, dst, value=w)

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

salida_authorrank = pd.read_excel("data/salida_AuthorRank.xlsx",index_col= 0)
salida_authorrank = salida_authorrank.sort_values(by='values', ascending=False)

norm = Normalize(vmin=salida_authorrank['values'].min(), vmax=salida_authorrank['values'].max())
mapper = ScalarMappable(norm=norm, cmap='Reds')

salida_authorrank['color'] = salida_authorrank['values'].apply(lambda x: mapper.to_rgba(x, bytes=True))
salida_authorrank['color'] = salida_authorrank['color'].apply(lambda c: f'#{c[0]:02x}{c[1]:02x}{c[2]:02x}')

print(salida_authorrank[['ins', 'values', 'nombre', 'color']])

       ins    values                              nombre    color
125   3970  0.002354                     Ibanez, Agustin  #67000c
741    133  0.001600                    Alfonso Urzúa M.  #df2c25
124     38  0.001439                    Adolfo M. Garcia  #f04130
1663  1231  0.001405                    Felipe E. García  #f14532
310   2750  0.001369                      Mariane Krause  #f34b36
...    ...       ...                                 ...      ...
3327   490  0.000016            Boris Valdenegro Egozcue  #fff5f0
9038  8382  0.000016         Aristegui, Daniela González  #fff5f0
7811  5192  0.000016                   Vargas, Nicole B.  #fff5f0
9032  8376  0.000016  Astorga, Paulina Soledad Santander  #fff5f0
9196  8707  0.000016            Cabezas Corcione, Andres  #fff5f0

[9393 rows x 4 columns]


In [16]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

for node in autor_net.nodes:

    n_art_autor = data_autores[node["id"]]['n_articulos']

    node["label"] = data_autores[node["id"]]["nombre_autor"]
    node["value"] = 100* n_art_autor
    node["title"] = data_autores[node["id"]]["nombre_autor"] + ", Número artículos: " + str(data_autores[node["id"]]["n_articulos"])
    node["color"] = salida_authorrank.loc[salida_authorrank['nombre'] == node["label"], 'color'].values[0]
    node["font"] = {"size":  n_art_autor, "color": "white"}

autor_net.show("GrafoCoautoria.html")

GrafoCoautoria.html
